In [3]:
import os

import numpy as np
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Import datasets

In [4]:
dir_csv = '../../data/bike_rental_history/raw'
list_csv = os.listdir(dir_csv)
unnecessary_columns = ['대여거치대', '반납거치대', '이용거리(M)', '이용시간(분)', '대여 대여소번호','반납대여소번호', '대여 대여소명', '반납대여소명']
dict_rename_columns={'자전거번호':'bike_id',
                     '대여일시':'rent_datetime',
                     '반납일시':'return_datetime',
                     '대여대여소ID':'origin_station_id',
                     '반납대여소ID':'desti_station_id',
                     '생년':'birth_year',
                     '이용자종류':'user_type',
                     '성별':'gender',
                     '자전거구분':'bike_type'}

# load holiday data
df_holiday = pd.read_csv('../../data/holidays.csv')
df_holiday.loc[:,'date'] = pd.to_datetime(df_holiday.loc[:,'year'].astype(str) +'-'+ df_holiday.loc[:,'month'].astype(str) +'-'+ df_holiday.loc[:,'day'].astype(str)).dt.date
df_holiday.loc[:,'date'] = df_holiday.loc[:,'date'].astype(str)
list_holiday = df_holiday.loc[:,'date'].unique().tolist()

## Merge datasets and drop some columns and values
list_df_bike = []
list_df_bike_weekday = []
list_df_bike_weekend = []
for csv in list_csv:
  df_tmp = pd.read_csv(f'{dir_csv}/{csv}', encoding='cp949')
  # drop the rows with missing values
  df_tmp = df_tmp.loc[np.logical_not(pd.isna(df_tmp.loc[:,'대여대여소ID']))]
  df_tmp = df_tmp.loc[np.logical_not(pd.isna(df_tmp.loc[:,'반납대여소ID']))]
  df_tmp = df_tmp.loc[np.logical_not(df_tmp.loc[:,'반납대여소ID'].str.contains('N'))]
  df_tmp.loc[:,'year'] = df_tmp.loc[:,'대여일시'].str[:4]
  df_tmp = df_tmp.loc[df_tmp.loc[:,'year'] == '2023']
  # drop unnecessary columns
  df_tmp = df_tmp.reset_index(drop=True).drop(columns=unnecessary_columns, axis=1)
  df_tmp = df_tmp.rename(columns=dict_rename_columns)
  df_tmp.loc[:,'rent_datetime'] = pd.to_datetime(df_tmp.loc[:,'rent_datetime'])
  df_tmp.loc[:,'weekday'] = pd.to_datetime(df_tmp.loc[:,'rent_datetime']).dt.weekday
  df_tmp.loc[:,'date'] = (pd.to_datetime(df_tmp.loc[:,'rent_datetime']).dt.date).astype(str)
  # split weekday and weekend (+ holiday)
  df_tmp_weekday = df_tmp.loc[np.logical_and(df_tmp.loc[:,'weekday'] < 5, ~df_tmp.loc[:,'date'].isin(list_holiday))]
  df_tmp_weekend = df_tmp.loc[np.logical_or(df_tmp.loc[:,'weekday'] >= 5, df_tmp.loc[:,'date'].isin(list_holiday))]
  # append the datasets
  list_df_bike.append(df_tmp)
  list_df_bike_weekday.append(df_tmp_weekday)
  list_df_bike_weekend.append(df_tmp_weekend)


# merge the imported datasets
df_bike_rental = pd.concat(list_df_bike, ignore_index=True)
df_bike_rental_weekday = pd.concat(list_df_bike_weekday, ignore_index=True)
df_bike_rental_weekend = pd.concat(list_df_bike_weekend, ignore_index=True)

In [5]:
df_bike_rental

,bike_id,rent_datetime,return_datetime,birth_year,gender,user_type,origin_station_id,desti_station_id,year,weekday,date,bike_type
0,SPB-39194,2023-01-01 00:02:13,2023-01-01 00:02:43,1977,\N,내국인,ST-2127,ST-2127,2023,6,2023-01-01,NaN
1,SPB-43457,2023-01-01 00:01:20,2023-01-01 00:03:28,1998,\N,내국인,ST-1121,ST-2699,2023,6,2023-01-01,NaN
2,SPB-44383,2023-01-01 00:04:01,2023-01-01 00:04:21,1993,M,내국인,ST-1590,ST-1590,2023,6,2023-01-01,NaN
3,SPB-37186,2023-01-01 00:02:09,2023-01-01 00:04:59,1965,M,내국인,ST-1841,ST-2740,2023,6,2023-01-01,NaN
4,SPB-59440,2023-01-01 00:02:24,2023-01-01 00:05:02,1986,M,내국인,ST-1215,ST-827,2023,6,2023-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
46196424,SPB-38642,2023-12-31 23:43:00,2024-01-01 03:28:36,2004,M,내국인,ST-321,ST-1011,2023,6,2023-12-31,일반자전거
46196425,SPB-52259,2023-12-31 17:47:40,2024-01-01 05:56:28,1994,M,내국인,ST-1812,ST-1812,2023,6,2023-12-31,일반자전거
46196426,SPB-50506,2023-12-31 19:31:15,2024-01-01 07:11:55,2000,M,내국인,ST-1263,ST-1194,2023,6,2023-12-31,일반자전거
46196427,SPB-30733,2023-12-31 22:13:05,2024-01-01 09:42:33,1967,M,내국인,ST-2031,ST-2890,2023,6,2023-12-31,일반자전거


In [6]:
df_bike_rental.to_csv('../../data/bike_rental_history/processed/2023_processed_entire.csv', index=False)
df_bike_rental_weekday.to_csv('../../data/bike_rental_history/processed/2023_processed_weekday.csv', index=False)
df_bike_rental_weekend.to_csv('../../data/bike_rental_history/processed/2023_processed_weekend.csv', index=False)

## Make a sample

In [7]:
df_bike_rental.sample(frac = 1/df_bike_rental.loc[:,'date'].nunique()).to_csv('../../data/bike_rental_history/processed_sample/2023_processed_entire_sample.csv', index=False)
df_bike_rental_weekday.sample(frac = 1/df_bike_rental_weekday.loc[:,'date'].nunique()).to_csv('../../data/bike_rental_history/processed_sample/2023_processed_weekday_sample.csv', index=False)
df_bike_rental_weekend.sample(frac = 1/df_bike_rental_weekend.loc[:,'date'].nunique()).to_csv('../../data/bike_rental_history/processed_sample/2023_processed_weekend_sample.csv', index=False)